In [1]:
# Import packages
import numpy as np
import ipdb
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

In [2]:
# Load training data, split into train and validation sets
data = pd.read_csv("train.csv")
train_data = data.sample(frac=0.8)
val_data = data.drop(train_data.index)
discrete_attributes = ["Work_accident", "promotion_last_5years", "sales", "salary"]    
real_attributes = ["satisfaction_level", "last_evaluation", "number_project", "average_montly_hours", "time_spend_company"]

In [3]:
def entropy_before_split(train_data):
    """ Finds Entropy of dataset before any split """
    
    dependent_variable = "left"
    count_0 = len(train_data[train_data[dependent_variable] == 0])
    total = len(train_data[dependent_variable])
    ratio = count_0 / total
    gini = 2 * (ratio) * (1 - ratio)
    return gini

In [4]:
def entropy_on_real_attribute_split(train_data,attribute):
    Class = "left" 
    class_labels = train_data[Class].unique()  #This gives all 'Yes' and 'No'
    attribute_labels = train_data[attribute].unique()  #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    class_labels.sort()
    attribute_labels.sort()
    combined = pd.DataFrame(train_data,columns=[attribute,'left']).sort_values(by=[attribute,'left'])
    
    split_node_gini = None
    split_node = None
    np_array_length = len(combined)
    total0 = len(combined[combined['left'] == 0])
    total1 = np_array_length - total0
    count0 = 0
    i = 0
    previous = None
    for row in combined.iterrows():
        if (i != 0 and previous != row[1][attribute]):
            gini = 0
            gini2 = 0
            num = count0 
            den = i
            temp = num / (den)
            gini += 2 * temp * (1 - temp)
            
            temp2 = den/np_array_length
            gini2 += temp2*gini
            
            gini = 0
            num = total0 - count0 #for > 0's
            den = np_array_length - i
            temp = num / (den+eps)
            gini += 2 * temp * (1 - temp)
            
            temp2 = den/np_array_length
            gini2 += temp2*gini
            
            if(split_node_gini == None):
                split_node_gini = abs(gini2)
                split_node = previous
            else:
                if(split_node_gini > abs(gini2)):
                    split_node_gini = abs(gini2)
                    split_node = previous
        if (row[1]['left']==0):
            count0 += 1
        previous = row[1][attribute]
        i += 1
        
    gini = 0
    gini2 = 0
    num = count0 #for <= 0's
    den = i
    temp = num / (den)
    gini += 2 * temp * (1 - temp)
    
    temp2 = den/np_array_length
    gini2 += temp2 * gini

    gini = 0
    num = total0 - count0 #for > 0's
    den = np_array_length - i
    temp = num / (den+eps)
    gini += 2 * temp * (1 - temp)
    
    temp2 = den/np_array_length
    gini2 += temp2 * gini

    if(split_node_gini == None):
        split_node_gini = abs(gini2)
        split_node = previous
    else:
        if(split_node_gini > abs(gini2)):
            split_node_gini = abs(gini2)
            split_node = previous
#     print (split_node,split_node_entropy)
    return split_node_gini, split_node

In [5]:
entropy_on_real_attribute_split(train_data, "satisfaction_level")

(0.2618944129979286, 0.46)

In [6]:
def entropy_on_discrete_attribute_split(train_data, attribute):
    """ Finds resulting entropy of dataset if it is split using attribute """
    gini_after_split = 0
    dependent_variable = "left"
    attribute_labels = train_data[attribute].unique()

    for attribute_label in attribute_labels:
        gini = 0
        numer = len(train_data[attribute][train_data[attribute] == attribute_label][train_data[dependent_variable] == 0])
        denom = len(train_data[attribute][train_data[attribute] == attribute_label])
        temp = numer / (denom + eps)
        gini = 2 * temp * (1 -  temp)
        temp2 = denom / (len(train_data) + eps)
        gini_after_split += temp2 * gini
    return abs(gini_after_split)

In [7]:
def split_criteria(train_data):
    """ Finds the best attribute to split on """
    
    Info_gain_discrete = {}
    initialEntropy = entropy_before_split(train_data)
    Info_gain = None
    split_point = None
    max_gain_attribute = None
    
    for key in discrete_attributes:
        Info_gain_discrete[key] = initialEntropy - entropy_on_discrete_attribute_split(train_data, key)
        
    for key in real_attributes:
        max_entropy, max_entropy_val = entropy_on_real_attribute_split(train_data, key)
        if (Info_gain == None or initialEntropy - max_entropy > Info_gain):
            Info_gain = initialEntropy - max_entropy
            split_point = max_entropy_val
            max_gain_attribute = key
             
    first = max(Info_gain_discrete, key=lambda k: Info_gain_discrete[k])
    if Info_gain_discrete[first] > Info_gain:
        Info_gain = Info_gain_discrete[first]
        max_gain_attribute = first
    return Info_gain, max_gain_attribute, split_point
    return Info_gain_discrete[first], first

In [8]:
def split_dataset_smaller(train_data, feature, split_point):
    return train_data[train_data[feature] <= split_point].reset_index(drop = True)

In [9]:
def split_dataset_greater(train_data, feature, split_point):
      return train_data[train_data[feature] > split_point].reset_index(drop = True)

In [10]:
def split_dataset(train_data, feature, label):
    """ This splits the dataset on given feature and all of its values """
    return train_data[train_data[feature] == label].reset_index(drop = True)

In [11]:
def most_probable(train_data):
    dependent_variable = "left"
    count_left = len(train_data[train_data[dependent_variable] == 0])
    count_right = len(train_data[train_data[dependent_variable] == 1])
    if count_left > count_right:
        return 0
    else:
        return 1

In [12]:
def Decision_tree(train_data):
    """ Builds tree recursively """
    
    D_tree = {}
    dependent_variable = "left"
    Info_gain, root, split_point = split_criteria(train_data)
#     print (Info_gain, root, split_point)
    if Info_gain == 0.0:
        return most_probable(train_data)
    
    D_tree[root] = {}
    
    if root in (discrete_attributes):
        labels = train_data[root].unique()
        for label in labels:
            split_data = split_dataset(train_data, root, label)
            unique_labels = split_data[dependent_variable].unique()
            if len(unique_labels) == 1:
                D_tree[root][label] = unique_labels[0]
            else:
                D_tree[root][label] = Decision_tree(split_data)
        return D_tree

    else:
        split_data = split_dataset_smaller(train_data, root, split_point)
        unique_labels = split_data[dependent_variable].unique()
        if len(unique_labels) == 1:
            D_tree[root][split_point] = unique_labels[0]
        else:
            D_tree[root][split_point] = Decision_tree(split_data)
            
        split_data = split_dataset_greater(train_data, root, split_point)
        unique_labels = split_data[dependent_variable].unique()
        if len(unique_labels) == 1:
            D_tree[root][split_point + 0.000000001] = unique_labels[0]
        else:
            D_tree[root][split_point + 0.000000001] = Decision_tree(split_data)
    
        return D_tree
    

In [13]:
def predict(inst, tree):
    for nodes in tree.keys():        
        value = inst[nodes]
        if nodes in discrete_attributes:
            if value in list((tree[nodes]).keys()):
                tree = tree[nodes][value]
            else:
                zeros = 0
                ones = 0
                for i in tree[nodes].keys():
                    if tree[nodes][i] == 0:
                        zeros += 1
                    elif tree[nodes][i] == 1:
                        ones += 1
                if zeros > ones:
                    return 0
                else:
                    return 1

        elif nodes in real_attributes:
            first_key = list(tree[nodes].keys())[0]
            if value <= first_key :
                tree = tree[nodes][first_key]
            else:
                second_key = list(tree[nodes].keys())[1]
                tree = tree[nodes][second_key]
        prediction = 0

        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;

    return prediction

In [14]:
# def predict(inst, tree):
#     #This function is used to predict for any input variable  
#     #Recursively we go through the tree that we built earlier
#     for nodes in tree.keys():        
#         value = inst[nodes]
#         if nodes in discrete_attributes:
#             if value not in tree.keys():
#                 count_left = 0
#                 count_right = 1
#                 for i in tree[nodes].values():
#                     if (type(i) != dict):
#                         if(i == 1):
#                             count_right += 1
#                         else:
#                             count_left += 0
#                 return max(count_left, count_right)
#             else:
#                 tree = tree[nodes][value]
#         else:
#             if (value <= list((tree[nodes]).keys())[0]):
#                 tree = tree[nodes][list((tree[nodes]).keys())[0]]
#             else:
#                 tree = tree[nodes][list((tree[nodes]).keys())[1]]
#         prediction = 0
#         if type(tree) is dict:
#             prediction = predict(inst, tree)
#         else:
#             prediction = tree
#             break;                            
        
#     return prediction

In [15]:
def validate_tree(val_data, tree):
    predicted = []
    for index, row in val_data.iterrows():
        predicted.append(predict(row, tree))
    actual = val_data["left"].tolist()
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    
    for i in range(0, len(predicted)):
        if (predicted[i] == 0 and actual[i] == 0):
            true_neg += 1
        elif (predicted[i] == 0 and actual[i] == 1):
            false_neg += 1
        elif (predicted[i] == 1 and actual[i] == 0):
            false_pos += 1
        else:
            true_pos += 1
    return true_pos, true_neg, false_pos, false_neg 
    
        

In [16]:
def accuracy(val_data, tree):
    true_pos, true_neg, false_pos, false_neg = validate_tree(val_data, tree)
#     print (true_pos, true_neg, false_pos, false_neg)
    total_instances = true_neg + true_pos + false_neg + false_pos
    accuracy_estimate = (true_neg + true_pos) / (total_instances)
    precision_estimate = true_pos / (true_pos + false_pos)
    recall_estimate = (true_pos) / (true_pos + false_neg)
    f1_score = (1 / recall_estimate) + (1 / precision_estimate)
    f1_score = 2 / f1_score
    print ("Accuracy : ", accuracy_estimate)
    print ("Precision : ", precision_estimate)
    print ("Recall : ", recall_estimate)
    print ("F1_Score : ", f1_score)

In [17]:
tree = Decision_tree(train_data)
accuracy(val_data, tree)

Accuracy :  0.7602313167259787
Precision :  0.490547263681592
Recall :  0.948076923076923
F1_Score :  0.6465573770491803
